In [143]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [144]:
print(tf.__version__)

2.8.0


In [145]:
import json
import string
import re
import gdown
SQLENGTH = 10
corpus = []
# read json file
fileName = 'Electronics_5.json'

writeFile = './Data.txt'
! rm './Data.txt'
# read
text = ""
f = open(writeFile, "a")
with open(fileName, "r") as read_file:
    i = 0
    l = read_file.readline()
    while l and i < 100000:
        if (i%1000 == 0):
            print(i)
        i+=1
        l_dict = json.loads(l)
        review = l_dict["reviewText"]
        r_list = review.split('.')
      
        for r in r_list:
            r = re.sub('[^a-zA-Z]', ' ', r)
            r = r.lower()
            r = r.split()
            r = [w for w in r if not w in set(string.punctuation)]
            j = 0
            while j+SQLENGTH < len(r):
                text = " ".join(r[j:j+10])
                corpus.append(text)
                f.write(text)
                f.write("\n")
                j+=SQLENGTH
            if j < len(r):
                text = " ".join(r[j:])
                corpus.append(text)
                f.write(text)
                f.write("\n")
        l = read_file.readline()
           
f.close()
print(text)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
programs or whatever


In [146]:
SEED = 42
num_ns = 4
AUTOTUNE = tf.data.AUTOTUNE
# path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
path_to_file = './Data.txt'

In [147]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))
# save file locally



In [148]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=SEED,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [149]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 10000
sequence_length = SQLENGTH

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [150]:
vectorize_layer.adapt(text_ds.batch(1024))

KeyboardInterrupt: 

In [ ]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'of', 'a', 'it', 'with', 'my', 'is', 'for', 'worked', 'within', 'when', 'very', 'use', 'up', 'unit', 'trucker', 'that', 'just']


In [ ]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [ ]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

18000


In [ ]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

18000


In [ ]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[36 75 41 74  9  7 71 32  8 28] => ['we', 'got', 'this', 'gps', 'for', 'my', 'husband', 'who', 'is', 'an']
[59 58  2 52 17  0  0  0  0  0] => ['otr', 'over', 'the', 'road', 'trucker', '', '', '', '', '']
[13 21  6  2 45 40  5 95  4 77] => ['very', 'impressed', 'with', 'the', 'shipping', 'time', 'it', 'arrived', 'a', 'few']
[84 82 42 78  0  0  0  0  0  0] => ['days', 'earlier', 'than', 'expected', '', '', '', '', '', '']
[11  4 35  3 14 72  5 44 24 15] => ['within', 'a', 'week', 'of', 'use', 'however', 'it', 'started', 'freezing', 'up']


In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 18000/18000 [00:03<00:00, 5286.15it/s]




targets.shape: (16878,)
contexts.shape: (16878, 5)
labels.shape: (16878, 5)


In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [ ]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [ ]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [ ]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [ ]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [ ]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
16/16 [==============================] - 1s 7ms/step - loss: 1.5954 - accuracy: 0.6246
Epoch 2/20
16/16 [==============================] - 0s 7ms/step - loss: 1.5531 - accuracy: 0.9401
Epoch 3/20
16/16 [==============================] - 0s 7ms/step - loss: 1.4830 - accuracy: 0.9524
Epoch 4/20
16/16 [==============================] - 0s 8ms/step - loss: 1.3690 - accuracy: 0.9511
Epoch 5/20
16/16 [==============================] - 0s 8ms/step - loss: 1.2067 - accuracy: 0.9496
Epoch 6/20
16/16 [==============================] - 0s 11ms/step - loss: 1.0086 - accuracy: 0.9486
Epoch 7/20
16/16 [==============================] - 0s 8ms/step - loss: 0.8032 - accuracy: 0.9486
Epoch 8/20
16/16 [==============================] - 0s 7ms/step - loss: 0.6220 - accuracy: 0.9487
Epoch 9/20
16/16 [==============================] - 0s 8ms/step - loss: 0.4824 - accuracy: 0.9498
Epoch 10/20
16/16 [==============================] - 0s 11ms/step - loss: 0.3835 - accuracy: 0.9516
Epoch 11/20
16/16

In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()